In [9]:
import ibis
from ibis_substrait.compiler.core import SubstraitCompiler
from icetrait.iceberg.process import ProcessSubstrait

In [10]:
ibis_tb = ibis.table([("id", "date"), ("bid", "float"), ("ask", "int32"), ("symbol", "string")], "t",)
query = ibis_tb.select(["id", "bid", "ask", "symbol"])
compiler = SubstraitCompiler()
protobuf_msg = compiler.compile(query).SerializeToString()

In [12]:
psb = ProcessSubstrait(protobuf_msg)

In [19]:
import duckdb

con = duckdb.connect()
con.install_extension("substrait")
con.load_extension("substrait")


In [25]:
con.execute(query='CREATE TABLE t1 (id int,bid float, ask int, symbol text);')
con.execute(query="INSERT INTO t1 VALUES (1, 0.1, 0.2, 'A'), (2, 0.2, 0.3, 'B');")

protobuf_msg = con.get_substrait(query="SELECT id as ID, bid as BID, ask as ASK, symbol AS SYMBOL FROM t1").fetchone()[0]    

In [26]:
psb = ProcessSubstrait(protobuf_msg)

relations {
  root {
    input {
      project {
        input {
          read {
            base_schema {
              names: "id"
              names: "bid"
              names: "ask"
              names: "symbol"
              struct {
                types {
                  i32 {
                    nullability: NULLABILITY_NULLABLE
                  }
                }
                types {
                  fp32 {
                    nullability: NULLABILITY_NULLABLE
                  }
                }
                types {
                  i32 {
                    nullability: NULLABILITY_NULLABLE
                  }
                }
                types {
                  varchar {
                    length: 1
                    nullability: NULLABILITY_NULLABLE
                  }
                }
                nullability: NULLABILITY_REQUIRED
              }
            }
            projection {
              select {
                struct_items {
     

In [28]:
query_result = con.from_substrait(proto=protobuf_msg)
query_result

┌───────┬───────┬───────┬─────────┐
│  ID   │  BID  │  ASK  │ SYMBOL  │
│ int32 │ float │ int32 │ varchar │
├───────┼───────┼───────┼─────────┤
│     1 │   0.1 │     0 │ A       │
│     2 │   0.2 │     0 │ B       │
└───────┴───────┴───────┴─────────┘

In [35]:
json_plan = con.get_substrait_json(query="SELECT id as ID, bid as BID, ask as ASK, symbol AS SYMBOL FROM t1").fetchone()[0]
print(json_plan)

{"relations":[{"root":{"input":{"project":{"input":{"read":{"baseSchema":{"names":["id","bid","ask","symbol"],"struct":{"types":[{"i32":{"nullability":"NULLABILITY_NULLABLE"}},{"fp32":{"nullability":"NULLABILITY_NULLABLE"}},{"i32":{"nullability":"NULLABILITY_NULLABLE"}},{"varchar":{"length":1,"nullability":"NULLABILITY_NULLABLE"}}],"nullability":"NULLABILITY_REQUIRED"}},"projection":{"select":{"structItems":[{},{"field":1},{"field":2},{"field":3}]},"maintainSingularStruct":true},"namedTable":{"names":["t1"]}}},"expressions":[{"selection":{"directReference":{"structField":{}},"rootReference":{}}},{"selection":{"directReference":{"structField":{"field":1}},"rootReference":{}}},{"selection":{"directReference":{"structField":{"field":2}},"rootReference":{}}},{"selection":{"directReference":{"structField":{"field":3}},"rootReference":{}}}]}},"names":["ID","BID","ASK","SYMBOL"]}}]}


### Projection Case Sample with Substrait

For column renames can be handled by just changing the `root.names` attribute.

Changes required for the Substrait plan

1. Update the field (field name or/and field type)
   a. If required we may have to do casting (type change)
2. Update `root.names` in `Plan.RelRoot`